# Load data


In [7]:
from function import *
import pandas as pd
import pandas as pd
import geopandas as gpd
from typing import Sequence



abandon_2d_variable = [
    "current_abandonment",
    "recultivation", 
    "abandonment_duration",
    "abandonment_year"
]
fea_3d_variable = [
    'GDPpc',
    'GDPtot',   
    'GURdist',
    'Population',
    'gdmp',
    'rsds',
    'tas',
    'wind'
]
fea_2d_variable = [
    'DEM',
    'Powerdist',
    'PrimaryRoad',
    'SecondaryRoad',
    'Slope',
    'TertiaryRoad'
]
PATHS = {
    'abandonment': r"D:\xarray\merged_chunk_2\*.nc",
    'feature':     "D:/xarray/aligned2/Feature_all/*.nc",
    'csv':         "aligned_for_training0519.csv",
    'prediction':  "",
    'prediction_us': r"data\abandon_filtered_with_scores.csv",
    'test_output': "positive_samples_test_500.csv",
    'output':      "positive_samples_full_with_features.csv",
    'CN_sheng': r'data\sheng2022.shp',
    'World_shp': r'data\main_ADM_0.shp', 
    'us_abandon': r'data\us_abandon_clean.csv',
    'us_pv_embedding': r'data\training_embedding.csv'
}

ZERO_COLS = [
     'GDPpc', 'GDPtot', 'GURdist', 'Population',
    'PrimaryRoad', 'SecondaryRoad', 'TertiaryRoad', 'gdmp'
]
YEARS = [2018, 2020]

NUMERIC_FEATURES = [
    'lat','lon','GDPpc', 'GDPtot', 'GURdist', 'DEM','Slope',
    'Population','Powerdist','PrimaryRoad','SecondaryRoad','TertiaryRoad',
    'gdmp','rsds','tas','wind'
]
CAT_COLS = ['landcover']

ABANDON_COLS = ['abandonment_year','abandonment_duration', 'current_abandonment']

NONE_ABANDON_COLS = ['recultivation']

time=['2018-01-01','2020-01-01']


us_df_abandon_filtered = pd.read_csv(PATHS['prediction_us'])
ds_abandon, ds_feat = load_datasets(PATHS['abandonment'], PATHS['feature'])
ds_merge=xr.merge([ds_abandon, ds_feat])









gogogo


KeyboardInterrupt: 

# 碳排放净减少效益测算

Given the'us_df_abandon_filtered' dataframe. 

| T_cell | 气温、辐射、风速 | a1+a2×Temperature+a3×Radiation+a4×Wind，其中a1=4.3℃，a2=0.0943（常数），a3=0.028℃/（辐射单位），a4=-1.528℃/（风速单位） | 这里将原文献仅用辐照数据的参数进一步改进 |
| ------ | -------- | ------------------------------------------------------------------------------------------------- | -------------------- |



输入数据：

将PV减排因子等价成对应能源系统的二氧化碳排放因子。

$$EF_{pv,mitigation}=E_{grid,CM}=EF_{grid,OM}W_{OM}+EF_{grid,BM}W_{BM}$$

一般来说，已经给出了各个国家的CM边际排放因子，可以直接沿用即可

us:352 gCO2/kWh


In [9]:
us_df_abandon_filtered

,Unnamed: 0,time,lat,lon,abandonment_year,abandonment_duration,current_abandonment,landcover,DEM,GDPpc,...,Powerdist,PrimaryRoad,SecondaryRoad,Slope,TertiaryRoad,gdmp,rsds,tas,wind,env_scores
0,2094,2020-01-01,25.004168,-107.545830,2018.0,6.0,1,2.0,128.7500,22847.0,...,622572.70,9.479545,2.584110,0.922605,44.558012,65.551445,20.555853,2981.984506,1.413225,0.305240
1,4586,2020-01-01,25.012500,-107.612500,2017.0,7.0,1,2.0,96.3125,11575.0,...,622572.60,7.480710,2.584110,4.124147,77.081042,76.545799,20.408821,2982.749690,1.381223,0.264706
2,4601,2020-01-01,25.012500,-107.487500,2017.0,7.0,1,2.0,176.3750,22847.0,...,622572.70,9.479545,2.584110,2.743563,51.958872,80.134598,20.785990,2980.215050,1.337188,0.293005
3,9586,2020-01-01,25.029167,-107.612500,2017.0,7.0,1,2.0,77.1250,11575.0,...,622572.60,7.480710,2.584110,1.319031,77.081042,62.175598,20.607885,2983.972465,1.395731,0.182346
4,9850,2020-01-01,25.029167,-105.412500,2017.0,7.0,1,2.0,1730.6250,10594.0,...,622573.30,7.372286,15.512376,1.524049,217.405949,104.869400,22.585236,2913.419221,1.931024,0.223332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70307,20522499,2020-01-01,48.187500,-65.837500,1998.0,26.0,1,3.0,46.5000,29640.0,...,622564.00,15.564339,0.007833,2.177264,91.230789,70.525002,12.947352,2763.749943,3.296368,0.307058
70308,20625641,2020-01-01,48.579166,-67.987500,1998.0,26.0,1,2.0,228.4375,33888.0,...,622562.94,54.122274,0.299834,1.650701,43.559419,58.398994,13.006569,2756.330329,4.223490,0.309734
70309,20669107,2020-01-01,48.745834,-72.437500,2012.0,12.0,1,2.0,105.2500,42697.0,...,622561.40,16.693830,1.095873,0.152130,40.225892,91.527328,12.529724,2756.581936,3.581519,0.306415
70310,20673507,2020-01-01,48.762500,-72.437500,2009.0,15.0,1,2.0,106.8125,43202.0,...,622561.40,8.394693,1.095873,0.207614,28.790137,79.406998,12.526788,2756.086073,3.699206,0.309188


# 1、方法A：光伏发电粗糙版思路

In [5]:
import pandas as pd
import numpy as np

def compute_pv_potential_fixed(
    df: pd.DataFrame,
    rsds_col: str = 'rsds',
    tas_col: str = 'tas',
    wind_col: str = 'wind',
    lat_col: str = 'lat',
    lom_col: str = 'lon',
    eta_stc: float = 0.17,
    eta_sys: float = 0.8,
    pixel_size_deg: float = 0.0083333
) -> pd.DataFrame:
    """
    修正后的思路A：假设 df['rsds'] 已经是 “物理上的 MJ/m²/d”。
    -------------------------------------------------------------
    1) I (W/m²) = df['rsds'] (MJ/m²/d) * 1e6 (J/MJ) / 86400 (s/d)
    2) T_amb_C (℃) = df['tas']*0.1 (从 K/10 到 K) - 273.15
    3) T_cell (℃) = 4.3 + 0.943*T_amb_C + 0.028*I - 1.528*wind
    4) PR = 1 + γ*(T_cell - 25), γ = -0.005 1/℃
    5) PVPOT = PR * I / 1000
    6) area_m2 = pixel 面积
    7) E_yr_per_m2 = η_stc * PVPOT * η_sys * 8760   (kWh/m²/yr)
    8) E_yr_pixel = E_yr_per_m2 * area_m2
    """
    df = df.copy()

    # ——（1）rsds: MJ/m²/d -> I: W/m² —— 
    # 直接用物理上已经是 MJ/m²/d 的数值（无需再乘 0.001）
    df['I'] = df[rsds_col] * 1e6 / 86400.0  # W/m²

    # ——（2）T_amb_C: tas (K/10) -> ℃ ——
    # 如果 NetCDF 原始里 tas 存成 K/10，需要先乘以 0.1
    df['T_amb_C'] = df[tas_col] * 0.1 - 273.15

    # ——（3）T_cell: 经验式 —— 
    df['T_cell'] = (
        4.3
        + 0.943 * df['T_amb_C']
        + 0.028 * df['I']
        - 1.528 * df['wind']
    )

    # ——（4）PR: 性能比 —— 
    gamma = -0.005
    df['PR'] = 1 + gamma * (df['T_cell'] - 25.0)

    # ——（5）PVPOT —— 
    df['PVPOT'] = df['PR'] * df['I'] / 1000.0

    # ——（6）像素面积 area_m2 —— 
    # 地球平均半径 ~ 6 371 000 m
    R = 6371000.0
    deg2rad = np.pi / 180.0
    dlat = pixel_size_deg * deg2rad
    dlon = pixel_size_deg * deg2rad
    lat_rad = df[lat_col] * deg2rad
    df['area_m2'] = (R * dlat) * (R * dlon * np.cos(lat_rad))

    # ——（7）单位面积年发电量 E_yr_per_m2 (kWh/m²/yr) —— 
    df['E_yr_per_m2'] = eta_stc * df['PVPOT'] * eta_sys * 8760.0

    # ——（8）像素总年发电量 E_yr_pixel (kWh/yr) —— 
    df['E_yr_pixel'] = df['E_yr_per_m2'] * df['area_m2']

    return df

# 用法示例（假设 df 已加载并含“rsds($\mathrm{MJ/m^2/d}$)”，“tas($\mathrm{K/10}$)”，“wind($\mathrm{m/s}$)”，“lat”，“lon”)：
fixed_df = compute_pv_potential_fixed(us_df_abandon_filtered)
# Calculate carbon emissions
fixed_df['carbon_emission (t)/yr'] = fixed_df['E_yr_pixel'] * 352 / 1000 / 1000


In [6]:
#fixed_df.to_csv('data/4.1_us_fixed.csv', index=False)
fixed_df.to_csv('data/4.data_prediction_net_benefit.csv', index=False)


# 2、方法B：逐月精细版（需要回到源数据提取colomn）

## 源数据增加

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def extract_monthly_rsds_fast(df, usa_bounds):
    """
    从每个月的 CHELSA rsds NetCDF 文件中批量提取指定点位的最近邻 rsds 值，
    并将结果写入 DataFrame。使用 xarray sel(method="nearest") 实现向量化查询，
    外层带 tqdm 进度条。

    参数
    ----
    df : pandas.DataFrame
        输入的点位表，必须包含 'lat' 和 'lon' 两列（浮点数），且每行表示一个点。
    usa_bounds : dict
        美国本土经纬度边框，格式如下：
            {
                'lat_min': ...,
                'lat_max': ...,
                'lon_min': ...,
                'lon_max': ...
            }

    返回
    ----
    pandas.DataFrame
        同原始 df 结构相同，但额外增加 12 列：
        'rsds_201901', 'rsds_201902', ..., 'rsds_201912'，分别对应 2019 年 1 至 12 月份
        在每个点位的 rsds 最近邻值。如果某个月子集为空则整列填 NaN。
    """

    # 复制一份，不改变原始 df
    result_df = df.copy()

    # 先把所有点的纬度和经度打包成 xarray.DataArray，维度名随意设为 "points"
    # 后面会在每个月做 sel(..., method="nearest") 时使用到
    lats_da = xr.DataArray(result_df["lat"].values, dims=("points",))
    lons_da = xr.DataArray(result_df["lon"].values, dims=("points",))

    # 用 tqdm 显示 month 进度条
    for month in tqdm(range(1, 13), desc="Processing months"):
        file_path = f'D:/xarray/aligned2/rsds/CHELSA_rsds_2019_{month:02d}_V.2.1.nc'

        try:
            with xr.open_dataset(file_path) as ds:
                # 打印原始经纬度范围，方便调试
                print(f"\n=== Month {month} ===")
                print("Original dataset ranges:")
                print(f"  lat: {ds.lat.min().values:.6f} to {ds.lat.max().values:.6f}")
                print(f"  lon: {ds.lon.min().values:.6f} to {ds.lon.max().values:.6f}")

                # 用 where+drop=True 做子集，避免坐标升降序的问题
                ds_subset = ds.where(
                    (ds.lat >= usa_bounds["lat_min"]) &
                    (ds.lat <= usa_bounds["lat_max"]) &
                    (ds.lon >= usa_bounds["lon_min"]) &
                    (ds.lon <= usa_bounds["lon_max"]),
                    drop=True
                )

                # 如果子集为空，则整列填 NaN 后直接继续下一个 month
                if ds_subset["rsds"].size == 0:
                    print(f"  Subset is empty for month {month}, filling NaN.")
                    result_df[f"rsds_2019{month:02d}"] = np.nan
                    continue

                # 打印子集范围，确保不为空
                print("Subset dataset ranges:")
                print(f"  lat: {ds_subset.lat.min().values:.6f} to {ds_subset.lat.max().values:.6f}")
                print(f"  lon: {ds_subset.lon.min().values:.6f} to {ds_subset.lon.max().values:.6f}")

                # 直接一次性按所有点做“最近邻”查询，返回维度为 ("points",) 的 DataArray
                rsds_at_points = ds_subset["rsds"].sel(
                    lat=lats_da,
                    lon=lons_da,
                    method="nearest"
                )

                # 将 DataArray 转为 numpy，然后赋值到 DataFrame 新列
                result_df[f"rsds_2019{month:02d}"] = rsds_at_points.values.astype(float)
                print(f"  Successfully extracted rsds for month {month}.")

        except Exception as e:
            # 任何报错时，该月全部填 NaN
            print(f"  Error processing month {month}: {e}")
            result_df[f"rsds_2019{month:02d}"] = np.nan

    return result_df


# ======================
# 使用示例
# ======================
if __name__ == "__main__":
    # 定义美国本土边框
    usa_bounds_main = {
        "lon_min": -125,
        "lon_max":  -65,
        "lat_min":   25,
        "lat_max":   49,
    }

    # 读取已有的点位 CSV，假设 PATHS['prediction_us'] 已经在外部定义
    us_df_abandon_filtered = pd.read_csv(PATHS["prediction_us"])
    print(f"Number of points to process: {len(us_df_abandon_filtered)}")

    # 调用函数
    result_df = extract_monthly_rsds_fast(us_df_abandon_filtered, usa_bounds_main)

    # 打印结果 DataFrame 信息
    print("\n最终 DataFrame 信息:")
    print(result_df.info())


Number of points to process: 70312


Processing months:   0%|          | 0/12 [00:00<?, ?it/s]


=== Month 1 ===
Original dataset ranges:
  lat: -89.995833 to 89.995833
  lon: -179.995833 to 179.995833
Subset dataset ranges:
  lat: 25.004167 to 48.995833
  lon: -124.995833 to -65.004167
  Successfully extracted rsds for month 1.

=== Month 2 ===
Original dataset ranges:
  lat: -89.995833 to 89.995833
  lon: -179.995833 to 179.995833
Subset dataset ranges:
  lat: 25.004167 to 48.995833
  lon: -124.995833 to -65.004167
  Successfully extracted rsds for month 2.

=== Month 3 ===
Original dataset ranges:
  lat: -89.995833 to 89.995833
  lon: -179.995833 to 179.995833
Subset dataset ranges:
  lat: 25.004167 to 48.995833
  lon: -124.995833 to -65.004167
  Successfully extracted rsds for month 3.

=== Month 4 ===
Original dataset ranges:
  lat: -89.995833 to 89.995833
  lon: -179.995833 to 179.995833
Subset dataset ranges:
  lat: 25.004167 to 48.995833
  lon: -124.995833 to -65.004167
  Successfully extracted rsds for month 4.

=== Month 5 ===
Original dataset ranges:
  lat: -89.995833 

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

def extract_monthly_rsds_fast(df, usa_bounds):
    

    # 复制一份，不改变原始 df
    result_df = df.copy()

    # 先把所有点的纬度和经度打包成 xarray.DataArray，维度名随意设为 "points"
    # 后面会在每个月做 sel(..., method="nearest") 时使用到
    lats_da = xr.DataArray(result_df["lat"].values, dims=("points",))
    lons_da = xr.DataArray(result_df["lon"].values, dims=("points",))

    # 用 tqdm 显示 month 进度条
    for month in tqdm(range(1, 13), desc="Processing months"):
        file_path = f'D:/xarray/aligned2/tas/CHELSA_tas_{month:02d}_2019_V.2.1.nc'

        try:
            with xr.open_dataset(file_path) as ds:
                # 打印原始经纬度范围，方便调试
                print(f"\n=== Month {month} ===")
                print("Original dataset ranges:")
                print(f"  lat: {ds.lat.min().values:.6f} to {ds.lat.max().values:.6f}")
                print(f"  lon: {ds.lon.min().values:.6f} to {ds.lon.max().values:.6f}")

                # 用 where+drop=True 做子集，避免坐标升降序的问题
                ds_subset = ds.where(
                    (ds.lat >= usa_bounds["lat_min"]) &
                    (ds.lat <= usa_bounds["lat_max"]) &
                    (ds.lon >= usa_bounds["lon_min"]) &
                    (ds.lon <= usa_bounds["lon_max"]),
                    drop=True
                )

                # 如果子集为空，则整列填 NaN 后直接继续下一个 month
                if ds_subset["tas"].size == 0:
                    print(f"  Subset is empty for month {month}, filling NaN.")
                    result_df[f"tas_2019{month:02d}"] = np.nan
                    continue

                # 打印子集范围，确保不为空
                print("Subset dataset ranges:")
                print(f"  lat: {ds_subset.lat.min().values:.6f} to {ds_subset.lat.max().values:.6f}")
                print(f"  lon: {ds_subset.lon.min().values:.6f} to {ds_subset.lon.max().values:.6f}")

                # 直接一次性按所有点做“最近邻”查询，返回维度为 ("points",) 的 DataArray
                tas_at_points = ds_subset["tas"].sel(
                    lat=lats_da,
                    lon=lons_da,
                    method="nearest"
                )

                # 将 DataArray 转为 numpy，然后赋值到 DataFrame 新列
                result_df[f"tas_2019{month:02d}"] = tas_at_points.values.astype(float)
                print(f"  Successfully extracted rsds for month {month}.")

        except Exception as e:
            print(f"  Error processing month {month}: {e}")
            # result_df[f"tas_2019{month:02d}"] = np.nan

    return result_df


# ======================
# 使用示例
# ======================
if __name__ == "__main__":
    # 定义美国本土边框
    usa_bounds_main = {
        "lon_min": -125,
        "lon_max":  -65,
        "lat_min":   25,
        "lat_max":   49,
    }

    # 读取已有的点位 CSV，假设 PATHS['prediction_us'] 已经在外部定义
    us_df_abandon_filtered = pd.read_csv(PATHS["prediction_us"])

    # 调用函数
    result_df = extract_monthly_rsds_fast(result_df, usa_bounds_main)

    # 打印结果 DataFrame 信息
    print("\n最终 DataFrame 信息:")
    print(result_df.info())


Processing months:   0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
result_df.to_csv('data/4.1_us_tas.csv', index=False)

# 3. Opportunity Loss


## 3.1 Step: Landchange Mask Map

Given ds_merge

it have time(year), lat, lon as coordinates and dimension.

It has varibles:
landcover, abandonment_year, etc.

For landcover, its float32, have the following values:

| New Class | Description |
|-----------|-------------|
| 1 | Cropland |
| 2 | Forest |
| 3 | Savanna |
| 4 | Shrub |
| 5 | Grassland and Arid Ecosystem |
| 6 | Wetland |
| 7 | Built-up |
| 8 | Bare Area and Ice |
| 9 | Water |

We want to get a new dataset, with same lat and lon, but without time dimension.


According to abandonment_year for each pixel, we want to get the landcover value for the year of abandonment.
For example, if abandonment_year is 2018, we want to get the landcover value for 2018.
and assign the landcover value to the corresponding pixel of new dataset.


In







Load Data

In [8]:
from function import *
import xarray as xr
ds_merge=load_all_ds()

ds_merge

c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 43
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 174
  result = blockwise(
c:\Users\zpy10\Anaconda3\envs\geo\Lib\site-packages\dask\array\core.py:4998: PerformanceWarning: Increasing number of chunks by factor of 14
  result = blockwise(
c:\Users\zpy10\Anacon

<xarray.Dataset> Size: 3TB
Dimensions:                   (time: 31, lat: 21600, lon: 43200)
Coordinates:
  * time                      (time) datetime64[ns] 248B 1992-01-01 ... 2022-...
    spatial_ref               int32 4B 0
    band                      int32 4B 1
  * lon                       (lon) float32 173kB -180.0 -180.0 ... 180.0 180.0
  * lat                       (lat) float32 86kB -90.0 -89.99 ... 89.99 90.0
Data variables: (12/20)
    abandonment_year          (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    abandonment_duration      (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    recultivation             (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    current_abandonment       (time, lat, lon) float32 116GB dask.array<chunksize=(31, 500, 496), meta=np.ndarray>
    landcover                 (time, lat, lon) float32 116GB dask.array<chunksize=(16, 250, 249), meta=np.ndarray>
    DEM                       (time, lat, lon) float32 116GB dask.array<chunksize=(31, 1440, 2880), meta=np.ndarray>
    ...                        ...
    rsds                      (time, lat, lon) float32 116GB dask.array<chunksize=(1, 3085, 6172), meta=np.ndarray>
    SecondaryRoad             (time, lat, lon) float64 231GB dask.array<chunksize=(31, 981, 1964), meta=np.ndarray>
    Slope                     (time, lat, lon) float32 116GB dask.array<chunksize=(31, 1440, 2880), meta=np.ndarray>
    tas                       (time, lat, lon) float64 231GB dask.array<chunksize=(1, 2700, 5400), meta=np.ndarray>
    TertiaryRoad              (time, lat, lon) float64 231GB dask.array<chunksize=(31, 981, 1964), meta=np.ndarray>
    wind                      (time, lat, lon) float32 116GB dask.array<chunksize=(1, 432, 500), meta=np.ndarray>

In [4]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm






# def get_landcover_at_abandonment(ds):
#     # 创建一个与landcover相同形状的结果数组
#     result = xr.full_like(ds.landcover.isel(time=0), np.nan)
    
#     # 对每个时间点进行处理
#     for t in ds.time:
#         # 获取当前时间点
#         current_time = t.values
#         current_year = pd.Timestamp(current_time).year
        
#         # 找到abandonment_year等于当前年份的像素
#         year_mask = (ds.abandonment_year.sel(time=current_time) == current_year)
        
#         # 提取这些像素对应的landcover值
#         landcover_values = ds.landcover.sel(time=current_time).where(year_mask)
        
#         # 更新结果，只在之前为nan的位置更新
#         result = xr.where(np.isnan(result) & ~np.isnan(landcover_values), 
#                          landcover_values, 
#                          result)
    
#     return result

# # 应用函数获取结果
# landcover_at_abandonment = get_landcover_at_abandonment(ds_merge)

# # 创建新的数据集
# ds_new = xr.Dataset({
#     'landcover_at_abandonment': landcover_at_abandonment
# })



In [ ]:
import netCDF4
import os
import gc
import psutil
from datetime import datetime
from numba import njit, prange, float32, int64
import numpy as np
import xarray as xr

@njit(float32[:,:](float32[:,:], float32[:,:,:], int64[:]), parallel=True)
def process_chunk_numba(abdt_years, landcover_vals, time_years):
    """
    计算每个像素在弃耕年份的耕地状态或众数
    """
    nlat, nlon = abdt_years.shape
    tlen = time_years.shape[0]
    out = np.full((nlat, nlon), np.nan, dtype=np.float32)

    uniq_vals = np.empty(tlen, dtype=np.float32)
    uniq_counts = np.zeros(tlen, dtype=np.int32)

    for i in prange(nlat):
        for j in prange(nlon):
            ay = abdt_years[i, j]
            if np.isnan(ay):
                continue
            idx = -1
            for t in range(tlen):
                if time_years[t] == ay:
                    idx = t
                    break
            if idx < 0:
                continue

            # 连续两年耕地检测
            consec = False
            for t in range(idx, tlen - 1):
                if landcover_vals[t, i, j] == 1 and landcover_vals[t+1, i, j] == 1:
                    out[i, j] = 1.0
                    consec = True
                    break
            if consec:
                continue

            # 弃耕后年份众数计算
            n_unique = 0
            max_count = 0
            mode_val = np.float32(np.nan)
            for t in range(idx, tlen):
                v = landcover_vals[t, i, j]
                if np.isnan(v):
                    continue
                found = False
                for k in range(n_unique):
                    if uniq_vals[k] == v:
                        uniq_counts[k] += 1
                        if uniq_counts[k] > max_count:
                            max_count = uniq_counts[k]
                            mode_val = v
                        found = True
                        break
                if not found:
                    uniq_vals[n_unique] = v
                    uniq_counts[n_unique] = 1
                    if 1 > max_count:
                        max_count = 1
                        mode_val = v
                    n_unique += 1
            out[i, j] = mode_val
    return out


def process_data_in_chunks(input_ds, output_path, chunk_size=500):
    """
    分块处理数据，避免大内存使用：
    - 使用传入的xarray Dataset而不是文件路径
    - 执行 NumPy/Numba 计算后写入并关闭输出文件

    Parameters:
    -----------
    input_ds: xarray.Dataset, 输入数据集，包含变量 'abandonment_year'和 'landcover'
    output_path: str, 输出 NetCDF 文件路径
    chunk_size: int, 块大小（像素数量）
    """
    def log_mem(msg=""):
        p = psutil.Process(os.getpid())
        mb = p.memory_info().rss / 1024**2
        pct = p.memory_percent()
        tot = psutil.virtual_memory().total / 1024**3
        print(f"[{datetime.now():%H:%M:%S}] {msg} 内存: {mb:.1f}MB ({pct:.1f}%/{tot:.1f}GB)")

    log_mem("开始处理前")

    # 获取维度
    nlat = input_ds.sizes['lat']
    nlon = input_ds.sizes['lon']
    time_years = input_ds['time'].dt.year.values.astype(np.int64)

    # 预编译 Numba 函数
    process_chunk_numba(
        np.zeros((1,1), np.float32),
        np.zeros((2,1,1), np.float32),
        np.array([2000,2001], np.int64)
    )

    total_blocks = int(np.ceil(nlat/chunk_size) * np.ceil(nlon/chunk_size))
    cnt = 0
    print("开始分块处理 …")

    # 初始化输出
    with netCDF4.Dataset(output_path, 'w') as dst:
        dst.createDimension('lat', nlat)
        dst.createDimension('lon', nlon)
        var = dst.createVariable('landcover_at_abandonment', 'f4', ('lat','lon'), zlib=True, complevel=1)
        var[:] = np.nan

    # 分块循环
    for i in range(0, nlat, chunk_size):
        i2 = min(i+chunk_size, nlat)
        for j in range(0, nlon, chunk_size):
            j2 = min(j+chunk_size, nlon)
            
            abd_da = input_ds['abandonment_year']
            if 'time' in abd_da.dims:
                abd_block = abd_da.isel(time=0, lat=slice(i,i2), lon=slice(j,j2))
            else:
                abd_block = abd_da.isel(lat=slice(i,i2), lon=slice(j,j2))
            abd = abd_block.values.astype(np.float32)

            lc = input_ds['landcover'].isel(lat=slice(i,i2), lon=slice(j,j2)).values.astype(np.float32)

            out_chunk = process_chunk_numba(abd, lc, time_years)

            # 写入并关闭
            with netCDF4.Dataset(output_path, 'a') as dst:
                dst.variables['landcover_at_abandonment'][i:i2, j:j2] = out_chunk

            # 清理
            del abd, lc, out_chunk
            gc.collect()

            cnt += 1
            log_mem(f"已处理 {cnt}/{total_blocks} 块")

    print("处理完成，输出：", output_path)

# 用法示例 - 直接使用Dataset而不是文件路径
process_data_in_chunks(ds_merge, 'D:/xarray/aligned2/emission/landcover_abd.nc', chunk_size=500)


ds_new=xr.open_dataset(r"D:\xarray\aligned2\emission\2_landcover_abd.nc")
# Get lat/lon coordinates from ds_merge
lat_coords = ds_merge.lat
lon_coords = ds_merge.lon
# Assign coordinates to ds_new
ds_new = ds_new.assign_coords(lat=lat_coords, lon=lon_coords).drop(['spatial_ref', 'band'])

[22:54:44] 开始处理前 内存: 2700.5MB (8.3%/31.7GB)
开始分块处理 …
[22:55:03] 已处理 1/3828 块 内存: 2705.3MB (8.3%/31.7GB)
[22:55:05] 已处理 2/3828 块 内存: 2705.4MB (8.3%/31.7GB)
[22:55:07] 已处理 3/3828 块 内存: 2704.5MB (8.3%/31.7GB)
[22:55:09] 已处理 4/3828 块 内存: 2704.5MB (8.3%/31.7GB)
[22:55:11] 已处理 5/3828 块 内存: 2706.4MB (8.3%/31.7GB)
[22:55:13] 已处理 6/3828 块 内存: 2707.7MB (8.3%/31.7GB)
[22:55:15] 已处理 7/3828 块 内存: 2707.8MB (8.3%/31.7GB)
[22:55:18] 已处理 8/3828 块 内存: 2707.8MB (8.3%/31.7GB)
[22:55:20] 已处理 9/3828 块 内存: 2707.8MB (8.3%/31.7GB)
[22:55:22] 已处理 10/3828 块 内存: 2707.8MB (8.3%/31.7GB)
[22:55:24] 已处理 11/3828 块 内存: 2707.9MB (8.3%/31.7GB)
[22:55:27] 已处理 12/3828 块 内存: 2707.9MB (8.3%/31.7GB)
[22:55:29] 已处理 13/3828 块 内存: 2708.8MB (8.3%/31.7GB)
[22:55:31] 已处理 14/3828 块 内存: 2706.0MB (8.3%/31.7GB)
[22:55:33] 已处理 15/3828 块 内存: 2707.0MB (8.3%/31.7GB)
[22:55:35] 已处理 16/3828 块 内存: 2707.0MB (8.3%/31.7GB)
[22:55:38] 已处理 17/3828 块 内存: 2707.0MB (8.3%/31.7GB)
[22:55:40] 已处理 18/3828 块 内存: 2708.0MB (8.3%/31.7GB)
[22:55:42] 已处理 19/38

In [15]:
ds_new.to_netcdf('data/4.landcover_abd.nc')
ds_new=xr.open_dataset('data/4.landcover_abd.nc')

In [10]:


# Convert float coordinates to integers for slicing
# ds_new.landcover_at_abandonment.sel(lon=slice(int(-96.66),int(-92.5)), lat=slice(int(27.5),int(31.66))).plot()
# Calculate and print number of valid pixels
# Count total valid pixels
valid_pixels = np.sum(~np.isnan(ds_new.landcover_at_abandonment))
print(f"Number of valid pixels: {valid_pixels}")

# Get histogram of values from 1-9
values, counts = np.unique(ds_new.landcover_at_abandonment.values[~np.isnan(ds_new.landcover_at_abandonment.values)], return_counts=True)
print("\nHistogram of landcover values:")
for val, count in zip(values, counts):
    print(f"Value {int(val)}: {count} pixels ({count/valid_pixels*100:.2f}%)")

# Plot the data
# ds_new.landcover_at_abandonment.plot()

Number of valid pixels: <xarray.DataArray 'landcover_at_abandonment' ()> Size: 4B
array(2004366)

Histogram of landcover values:
Value 1: 35558 pixels (1.77%)
Value 2: 1571769 pixels (78.42%)
Value 3: 170931 pixels (8.53%)
Value 4: 110731 pixels (5.52%)
Value 5: 66440 pixels (3.31%)
Value 6: 31267 pixels (1.56%)
Value 7: 8539 pixels (0.43%)
Value 8: 3002 pixels (0.15%)
Value 9: 6129 pixels (0.31%)


## 3.2 Generate Probability Map

Given ds_new, it has lat, lon, landcover_at_abandonment.

Most of pixels are nan, it has unique values: 1,2,3,4,5,6,7,8,9.


1. We want to Generate density map for each value, as new variable, counts_1- counts_9


Logic 如下，

1. Given value (eg 2)
Generate a mask ==2
每10个grid合并， 然后新的值aggregate为 counts.





2. We want to generate a weighted density map for each value, as new variable, weighted_density_1, weighted_density_2, weighted_density_3, weighted_density_4, weighted_density_5, weighted_density_6, weighted_density_7, weighted_density_8, weighted_density_9.
    （The sum of (weighted density from 1-9 for each pixel value is 1)







In [ ]:
import xarray as xr
import numpy as np
from tqdm.auto import tqdm
ds_new=xr.open_dataset('data/4.landcover_abd.nc')
# First check if ds_new has the expected structure
print("ds_new structure:")
print(ds_new)

def calculate_counts(ds, value=None, grid_size=10):
    """
    Calculate grid counts
    value: int or None. If None, calculate sum of all values (1-9); if int, calculate count for specific value
    """
    if value is None:
        # Calculate total counts for all values
        mask = (ds.landcover_at_abandonment >= 1) & (ds.landcover_at_abandonment <= 9)
        mask = mask.astype(float)
    else:
        # Calculate counts for specific value
        mask = (ds.landcover_at_abandonment == value).astype(float)
    
    # Aggregate grid
    counts = mask.coarsen(
        lon=grid_size,
        lat=grid_size,
        boundary='trim'
    ).sum()
    
    return counts

def calculate_weighted_densities(ds):
    """Calculate weighted densities, return new dataset"""
    # Create new dataset to store results
    ds_density = xr.Dataset()
    
    # First calculate total counts
    total_counts = calculate_counts(ds, value=None)
    ds_density['total_counts'] = total_counts

    # Calculate weighted density for each value
    for value in tqdm(range(1, 10), desc='Calculating weighted densities'):
        counts = calculate_counts(ds, value)
        # Calculate and store weighted density
        weighted_density = counts / total_counts
        ds_density[f'weighted_density_{value}'] = weighted_density
    
    return ds_density

try:
    # Try using the functions
    ds_density = calculate_weighted_densities(ds_new)
    
    # Save in chunks to avoid memory error
    ds_density.to_netcdf(
        r"data\3_ds_density.nc",
        encoding={var: {'zlib': True, 'complevel': 1} for var in ds_density.data_vars}
    )
    
    print("Successfully processed and saved density data")
    
except Exception as e:
    print(f"Error occurred: {str(e)}")


ds_new structure:
<xarray.Dataset> Size: 4GB
Dimensions:                   (lat: 21600, lon: 43200)
Coordinates:
  * lat                       (lat) float32 86kB -90.0 -89.99 ... 89.99 90.0
  * lon                       (lon) float32 173kB -180.0 -180.0 ... 180.0 180.0
Data variables:
    landcover_at_abandonment  (lat, lon) float32 4GB ...


Calculating weighted densities: 100%|██████████| 9/9 [03:30<00:00, 23.41s/it]


Successfully processed and saved density data


## 3.3 生成概率表格

In [14]:
import xarray as xr
import numpy as np
from tqdm.auto import tqdm

ds_density = xr.open_dataset(r"data\3_ds_density.nc")
# # Create new dataset with float32 coordinates
# ds_mergeds_density = ds_density.copy()
# ds_mergeds_density = ds_mergeds_density.assign_coords({
#     'lon': ds_mergeds_density.lon.astype('float32'),
#     'lat': ds_mergeds_density.lat.astype('float32')
# })

# Extract all variables to dataframe
df_density = ds_density.to_dataframe().reset_index()
df_density = df_density.dropna(subset=['weighted_density_1'])
df_density.to_csv('data/4.1_density.csv', index=False)
 


## 3.4 Interpolate

In [ ]:

Given table, df_density


df_prediction,  


基于kd tree, 插值
add column: 1-9 weighted_density



In [13]:
df_density

,lat,lon,total_counts,weighted_density_1,weighted_density_2,weighted_density_3,weighted_density_4,weighted_density_5,weighted_density_6,weighted_density_7,weighted_density_8,weighted_density_9
1841663,-54.458332,-68.041672,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1850295,-54.291668,-68.708328,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1850302,-54.291668,-68.125000,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1854632,-54.208332,-67.291672,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1854634,-54.208332,-67.125000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8362668,71.291672,109.041672,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8366888,71.375000,100.708328,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8366889,71.375000,100.791672,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8366956,71.375000,106.375000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.5 Compute Expectation



For Value 1: xxxx

公式
where, alpha is wind speed (feature)

For xxx

In [ ]:
全是1的碳排放


全是2的 , 

etc,.

这里你可以用到特征，如果没有特征就完了
Get E(Y)

## 3.6结束

a-E(Y)